In [ ]:
import plotly.express as px

from core.pandas_utils import *
from transform.processed_tss.ProcessedTimeSeries import ProcessedTimeSeries

In [ ]:
tss = pd.read_parquet("tesla_tss.parquet") #ProcessedTimeSeries("tesla")

In [ ]:
vin_with_most_charges = tss.groupby("vin")["in_charge_idx"].max().idxmax()
ts = tss.query("vin == @vin_with_most_charges")

In [ ]:
px.scatter(
    ts,
    x='date',
    y='soc',
    color="charger_power",
)

In [ ]:
tss.groupby("vin")["charger_power"].max().sort_values()

In [ ]:
vin_with_highest_charger_power = tss.groupby("vin")["charger_power"].max().idxmax()
ts = tss.query("vin == @vin_with_highest_charger_power")

In [ ]:
px.scatter(
    tss.query("vin == '5YJ3E7EB7LF568267' & charger_power > 0"),
    x='date',
    y='soc',
    color="charger_power",
    color_continuous_scale="Rainbow",
)

In [ ]:
charges:DF = (
    tss
    .query("trimmed_in_charge")
    .groupby(["vin", "trimmed_in_charge_idx"])
    .agg(
        count=pd.NamedAgg("charger_power", "count"),
        charger_power=pd.NamedAgg("charger_power", "max"),
    )
    .eval("charging_level = 'level_1'")
    .eval("charging_level = charging_level.mask(charger_power.between(2.5, 20) , 'level_2')")
    .eval("charging_level = charging_level.mask(charger_power > 20, 'level_3')")
    .sort_values("charger_power", ascending=False)
)
charges

In [ ]:
charges["count"].value_counts(sort=True, ascending=False, dropna=False, normalize=True)

In [ ]:
charges["charging_level"].value_counts(sort=True, ascending=False, dropna=False, normalize=True)

In [ ]:
px.histogram(
    charges,
    x="count",
)

In [ ]:
ts = tss.query("vin == '7SAXCCE5XRF452132'")
px.scatter(
    ts,
    x='date',
    y='soc',
    color="charger_power",
    color_continuous_scale="Rainbow",
)